# 前処理

- import 
- ファイル呼び出し
- ファイルの中身確認

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_profiling as pdp

In [12]:
FILE_PATH = '/Users/atsushisato/Google_drive/05_PBL/second_section/practice_4/'

coupon = pd.read_csv(FILE_PATH + 'coupon.csv')
customer =  pd.read_csv(FILE_PATH + 'customer.csv')
history =  pd.read_csv(FILE_PATH + 'purchase_history.csv')

In [13]:
#データの整形

#customer ===================================================
#カラム名変更
rename_dict = {'顧客ID':'uuid', '年齢層':'age_range', '既婚・独身':'marry', '家族人数':'fammily_num',
               '子供人数':'child_num', '持家':'house_flg', '年収レベル':'income'}
customer = customer.rename(columns=rename_dict)

#欠損値補完
customer['marry'] = customer['marry'].fillna('無回答')
customer['child_num'] = customer['child_num'].fillna('0')
customer['child_num'] = customer['child_num'].apply(lambda x: int(x.replace("+", "")))
customer['fammily_num'] = customer['fammily_num'].apply(lambda x: int(x.replace("+", "")))

#history ===================================================
#カラム名変更
history = history.rename(columns={'顧客ID':'uuid'})

#coupon ===================================================
#カラム名変更
rename_dict = {'クーポンID':'coupon_id','商品カテゴリ':'category', '商品カテゴリID':'category_id'}
coupon = coupon.rename(columns=rename_dict)


#カテゴリ別のカラム（購入回数・購入商品数・クーポン割引）に再作成
tmp_list = []

for category_id in coupon['category_id'].unique():
    column_list = ['uuid', '来店頻度', '経過日数', '総購入金額']
    column_list.extend([column for column in history.columns if category_id + '_' in column ])
    
    tmp_df = history[column_list]
    rename_dict = {category_id+'_購入回数':'purchase_num', category_id+'_購入商品数':'purchase_product_num',
                   category_id+'_クーポン割引':'coupon_use', category_id+'_購入金額':'purchase_price',
                   '来店頻度':'visits_frequency', '経過日数':'pass_days', '総購入金額':'all_purchase_price'}
    tmp_df = tmp_df.rename(columns=rename_dict)
    
    tmp_df['coupon_id'] = int(category_id.replace('c',''))
    tmp_list.append(tmp_df)    
history = pd.concat(tmp_list)

customer_df = pd.merge(customer, history, on='uuid', how='inner')
customer_df = pd.merge(customer_df, coupon, on='coupon_id', how='inner')

customer_df = customer_df.sort_values(['uuid','coupon_id']).reset_index(drop=True)
customer_df.head()

,uuid,age_range,marry,fammily_num,child_num,house_flg,income,visits_frequency,pass_days,all_purchase_price,purchase_num,purchase_product_num,coupon_use,purchase_price,coupon_id,category_id,category
0,1,40-49,無回答,1,0,0,5,9,19,56562,1,2,0,534,1,c1,米・パン
1,1,40-49,無回答,1,0,0,5,9,19,56562,1,2,0,2803,2,c2,肉
2,1,40-49,無回答,1,0,0,5,9,19,56562,0,0,0,0,3,c3,海鮮
3,1,40-49,無回答,1,0,0,5,9,19,56562,1,1,0,764,4,c4,自然食品
4,1,40-49,無回答,1,0,0,5,9,19,56562,9,17,0,4981,5,c5,即席食品


In [14]:
customer_df.to_csv(FILE_PATH + 'customer_df.csv',index=False)